In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import csv
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from utils.constants.dir_path import DataDirPath
from utils.constants.constants import Constants
from utils.convert_csv import ConvertCsv

In [7]:
base_info = DataDirPath.base_structure_info()
dir_path = '/Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data'
dirs = glob.glob(f'{dir_path}/*')

In [8]:
PATH = Constants.path()
base_df = pd.read_csv(f'{base_info}/base_info.csv')
base_df

,structure,structure_idx,natom,E,Vol
0,alpha-critobalite,1,48,-1011.449548,839.055430
1,alpha-critobalite,2,48,-1011.449396,839.055430
2,alpha-critobalite,3,48,-1011.449194,839.055430
3,alpha-critobalite,4,48,-1011.448339,839.055430
4,alpha-critobalite,5,48,-1011.448402,839.055430
...,...,...,...,...,...
8545,stishovite,696,48,-1010.442985,331.977432
8546,stishovite,697,48,-1010.386112,331.977432
8547,stishovite,698,48,-1010.322835,331.977432
8548,stishovite,699,48,-1010.398489,331.977432


In [79]:
# 各構造ごとのstructure_idxを全構造のindexに変更する
plus = 0
for structure in PATH:
    structure = structure.split('_')[-1]
    base_df.loc[base_df.structure == structure, ['structure_idx']] += plus
    idx = base_df[base_df.structure == structure].structure_idx.unique()
    idx.sort()
    plus = idx[-1]
    print(structure,idx[0],idx[-1])

alpha-critobalite 1 1150
alpha-quartz 1151 2250
beta-quartz 2251 3050
beta-trydymite 3051 4300
coesite 4301 5500
Fdd2-beta-critobalite 5501 6850
hex-trydymite 6851 7850
stishovite 7851 8550


In [80]:
dirs.sort()
with open('/Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/error.csv','a') as f:
    writer = csv.writer(f)
    writer.writerow(['type','structure','r-cut','num-pairs','MAE','MSE','RMSE','R2'])
for directory in dirs:
    files = glob.glob(f'{directory}/*')

    try:
        test_file = glob.glob(f'{directory}/test*')[0].split('/')[-1]
        train_file = glob.glob(f'{directory}/train*')[0].split('/')[-1]
    except Exception as e:
        print(f'no file in {directory}')
        continue

    obj = ConvertCsv(directory,directory)
    obj.convert_n2p2_output_to_csv(10,test_file,'tmp_test')
    obj.convert_n2p2_output_to_csv(10,train_file,'tmp_train')

    df_test = pd.read_csv(f'{directory}/tmp_test.csv')
    df_train = pd.read_csv(f'{directory}/tmp_train.csv')
    # テストに対して
    df_test_new = pd.merge(df_test,base_df,left_on='index',right_on='structure_idx')

    mae_test = mean_absolute_error(df_test_new['Eref'],df_test_new['Ennp'])
    mse_test = mean_squared_error(df_test_new['Eref'],df_test_new['Ennp'])
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(df_test_new['Eref'],df_test_new['Ennp'])

    r_cut = directory.split('/')[-1].split('_')[1]
    num_pairs = directory.split('/')[-1].split('_')[2]

    with open('/Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/error.csv','a') as f:
        writer = csv.writer(f)
        writer.writerow(['test','all',r_cut,num_pairs,mae_test,mse_test,rmse_test,r2_test])
    # 訓練データに対して
    df_train_new = pd.merge(df_train,base_df,left_on='index',right_on='structure_idx')

    mae_train = mean_absolute_error(df_train_new['Eref'],df_train_new['Ennp'])
    mse_train = mean_squared_error(df_train_new['Eref'],df_train_new['Ennp'])
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(df_train_new['Eref'],df_train_new['Ennp'])

    r_cut = directory.split('/')[-1].split('_')[1]
    num_pairs = directory.split('/')[-1].split('_')[2]

    with open('/Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/error.csv','a') as f:
        writer = csv.writer(f)
        writer.writerow(['train','all',r_cut,num_pairs,mae_train,mse_train,rmse_train,r2_train])

no file in /Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data/nnp-train_10_19
no file in /Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data/nnp-train_10_20
no file in /Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data/nnp-train_5_13
no file in /Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data/nnp-train_5_14
no file in /Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data/nnp-train_5_15
no file in /Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data/nnp-train_5_16
no file in /Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data/nnp-train_5_17
no file in /Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data/nnp-train_5_18
no file in /Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data/nnp-train_5_19
no file in /Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data/nnp-train_5_20
no file in /Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/scp_data/nnp-train_6_14
no file in /Users/y1u0d2/Desktop/Lab/resu

In [5]:
df = pd.read_csv('/Users/y1u0d2/Desktop/Lab/result/nnp-train/two-body/error.csv')
df[df['r-cut'] == 5]


,type,structure,r-cut,num-pairs,MAE,MSE,RMSE,R2
212,test,all,5,10,0.083754,0.014102,0.118753,0.986314
213,train,all,5,10,0.180466,21.517466,4.638692,-20.594041
214,test,all,5,11,0.074392,0.010984,0.104803,0.989340
215,train,all,5,11,0.120237,10.927551,3.305685,-9.966439
216,test,all,5,12,0.077606,0.011774,0.108508,0.988573
217,train,all,5,12,0.567134,1142.230859,33.796906,-1145.295724
218,test,all,5,3,0.121432,0.030012,0.173239,0.970873
219,train,all,5,3,0.122983,0.029132,0.170681,0.970764
220,test,all,5,4,0.112975,0.021600,0.146969,0.979037
221,train,all,5,4,0.114865,0.021472,0.146532,0.978452
